# UTILITIES

## SHELL OUTPUT

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## MODULES IMPORT

In [ ]:
import os

import tensorflow as tf
import numpy as np
from PIL import Image

SEED = 1234
tf.random.set_seed(SEED) 

## UNZIP DATA

In [ ]:
!wget "https://competitions.codalab.org/my/datasets/download/29a85805-2d8d-4701-a9ab-295180c89eb3"
!unzip -q /content/29a85805-2d8d-4701-a9ab-295180c89eb3

# DIRECTORIES DEFINITION

## CROPPING FUNCTION

In [ ]:
import cv2

def img_crop(img_source, img_dest, mask_source, mask_dest, name, frmt):

  """
  This function crops a specific image into 256x256 pieces. If the image is not
  divisible by 256, it will create some images with duplicated portions.

  """

  DIM = 256

  update_last_row = False
  update_last_column = False

  img_arr = cv2.imread(img_source)
  mask_arr = cv2.imread(mask_source)

  if (img_arr.shape[0]%DIM):
    update_last_row = True
  if (img_arr.shape[1]%DIM):
    update_last_column = True

  # default crops
  for l in range(img_arr.shape[0]//DIM):
    for c in range(img_arr.shape[1]//DIM):
      # image crop & save
      crop_img_arr = img_arr[l*DIM:l*DIM+DIM, c*DIM:c*DIM+DIM]
      cv2.imwrite(os.path.join(img_dest, name + "_" + str(l) + "_" + str(c) + frmt), crop_img_arr)
      # mask crop & save
      crop_mask_arr = mask_arr[l*DIM:l*DIM+DIM, c*DIM:c*DIM+DIM]
      cv2.imwrite(os.path.join(mask_dest, name + "_" + str(l) + "_" + str(c) + '.png'), crop_mask_arr)
  
  # last smaller column crop (there will be some duplicated portions of images)
  if update_last_column:
    for l in range(img_arr.shape[0]//DIM):
      # image crop & save
      crop_img_arr = img_arr[l*DIM:l*DIM+DIM, img_arr.shape[1]-DIM:img_arr.shape[1]]
      cv2.imwrite(os.path.join(img_dest, name + "_" + str(l) + "_last" + frmt), crop_img_arr)
      # mask crop & save
      crop_mask_arr = mask_arr[l*DIM:l*DIM+DIM, img_arr.shape[1]-DIM:img_arr.shape[1]]
      cv2.imwrite(os.path.join(mask_dest, name + "_" + str(l) + "_last" + '.png'), crop_mask_arr)

  # last smaller row crop (there will be some duplicated portions of images)
  if update_last_row:
    for c in range(img_arr.shape[1]//DIM):
      # image crop & save
      crop_img_arr = img_arr[img_arr.shape[0]-DIM:img_arr.shape[0], c*DIM:c*DIM+DIM]
      cv2.imwrite(os.path.join(img_dest, name + "_last" + "_" + str(c) + frmt), crop_img_arr)
      # mask crop & save
      crop_mask_arr = mask_arr[img_arr.shape[0]-DIM:img_arr.shape[0], c*DIM:c*DIM+DIM]
      cv2.imwrite(os.path.join(mask_dest, name + "_last" + "_" + str(c) + '.png'), crop_mask_arr)

  # remaining portion crop (there will be some duplicated portions of images)
  if update_last_column and update_last_row:
    # image crop & save
    crop_img_arr = img_arr[img_arr.shape[0]-DIM:img_arr.shape[0], img_arr.shape[1]-DIM:img_arr.shape[1]]
    cv2.imwrite(os.path.join(img_dest, name + "_last_last" + frmt), crop_img_arr)
    # mask crop & save
    crop_mask_arr = mask_arr[img_arr.shape[0]-DIM:img_arr.shape[0], img_arr.shape[1]-DIM:img_arr.shape[1]]
    cv2.imwrite(os.path.join(mask_dest, name + "_last_last" + '.png'), crop_mask_arr)


## FINAL DATASET
This script will create a folder called "**FinalDataset**" and a sub-folder called "training".

### training dir

Inside "**training**" there will be again 2 sub-folders: "images" and "masks". The first one will contain all the four-team images depicting both mais and haricot. On the other side, the second sub-directory will contain the respective masks of the pictures present in "images".

In [ ]:
import shutil

dataset_dir = '/content/FinalDataset'
training_dir = os.path.join(dataset_dir, 'training')

if not os.path.exists(dataset_dir):
  
  # FinalDataset
  os.makedirs(dataset_dir)

  # training
  os.makedirs(training_dir)
  # images & masks
  os.makedirs(os.path.join(training_dir, 'images/img'))
  os.makedirs(os.path.join(training_dir, 'masks/img'))

  starting_training_dir = '/content/Development_Dataset/Training'
  for team in os.listdir(starting_training_dir):
    for crop in os.listdir(os.path.join(starting_training_dir, team)):
      for f in os.listdir(os.path.join(starting_training_dir, team, crop, 'Images')):
        img_crop(img_source=os.path.join(starting_training_dir, team, crop, 'Images', f),
                 img_dest=os.path.join(training_dir, 'images/img'),
                 mask_source=os.path.join(starting_training_dir, team, crop, 'Masks', f[:-4] + '.png'),
                 mask_dest=os.path.join(training_dir, 'masks/img'),
                 name=f[:-4],
                 frmt=f[-4:])

### validation dir
This script will divide the whole content of "/content/FinalDataset/training/images" into 2 parts. The 80% of images into "training" and the 20% into "**validation**".

In [ ]:
import random, itertools 

validation_dir = '/content/FinalDataset/validation'

if not os.path.exists(validation_dir):
  os.makedirs(validation_dir)
  os.makedirs(os.path.join(validation_dir, 'images/img'))
  os.makedirs(os.path.join(validation_dir, 'masks/img'))
else:
  # if already present the validation dir, this script will shuffle its content. 
  # It will moove all the elements again to the "training" dir and after a shuffle 
  # it will take 20% of the content and put ot in the "validation" dir.
  for f in os.listdir(os.path.join(validation_dir, 'images/img')):
      o = shutil.move(os.path.join(validation_dir, 'images/img', f),
                      os.path.join(training_dir, 'images/img', f))
      o = shutil.move(os.path.join(validation_dir, 'masks/img', f[:-3] + 'png'),
                      os.path.join(training_dir, 'masks/img', f[:-3] + 'png'))
      
    
images = os.listdir(os.path.join(training_dir, 'images/img'))
random.shuffle(images)
for f in itertools.islice(images, 0, int(0.2*len(images))):
  o = shutil.move(os.path.join(training_dir, 'images/img', f),
                  os.path.join(validation_dir, 'images/img', f))
  o = shutil.move(os.path.join(training_dir, 'masks/img', f[:-3] + 'png'),
                  os.path.join(validation_dir, 'masks/img', f[:-3] + 'png'))

In [ ]:
!ls /content/FinalDataset/validation/img/images | wc -l
#!rm -rf ./FinalDataset

# GENERATORS

## IMAGE DATA GENERATOR

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=30,
                                            width_shift_range=10,
                                            height_shift_range=10,
                                            zoom_range=0.3,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='reflect',
                                            rescale=1./255)
    train_mask_data_gen = ImageDataGenerator(rotation_range=30,
                                             width_shift_range=10,
                                             height_shift_range=10,
                                             zoom_range=0.3,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='reflect')
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255)
    train_mask_data_gen = ImageDataGenerator()
    
valid_img_data_gen = ImageDataGenerator(rescale=1./255)
valid_mask_data_gen = ImageDataGenerator()

## FLOW FROM DIRECTORY

In [ ]:
# img shape
img_h = 256
img_w = 256

# Batch size
bs = 64

train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs,
                                                       class_mode=None, 
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED) 

train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None,
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         seed=SEED)
train_gen = zip(train_img_gen, train_mask_gen)

valid_img_gen = valid_img_data_gen.flow_from_directory(os.path.join(validation_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, 
                                                       shuffle=False,
                                                       interpolation='bilinear',
                                                       seed=SEED)
valid_mask_gen = valid_mask_data_gen.flow_from_directory(os.path.join(validation_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs, 
                                                         class_mode=None,
                                                         shuffle=False,
                                                         interpolation='bilinear',
                                                         seed=SEED)
valid_gen = zip(valid_img_gen, valid_mask_gen)

## PREPARE TARGET FUNCTION
Questa funzione serve per fare in modo che le maschere non siano poi immagini RGB ma immagini a singolo canale con i target corrispondenti alle classi del problema.

In [ ]:
def prepare_target(x_, y_):
  # crop
  y_1 = tf.where(tf.reduce_any(y_ == [255, 255, 255], axis=-1, keepdims=True), 1, 0)
  print(y_1.shape)
  # weed
  y_2 = tf.where(tf.reduce_any(y_ == [216, 67, 82], axis=-1, keepdims=True), 2, 0)
  return x_, tf.cast(y_1+y_2, tf.float32)

## DATASET DEFINITION

In [ ]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 3]))
train_dataset = train_dataset.map(prepare_target)
train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 3]))
valid_dataset = valid_dataset.map(prepare_target)
valid_dataset = valid_dataset.repeat()

## DATA GENERATOR TEST

In [ ]:
iterator = iter(valid_dataset)

In [ ]:
import time
import matplotlib.pyplot as plt

%matplotlib inline

# Assign a color to each class
colors_dict = {}
colors_dict[1] = [255, 255, 255]  # crop
colors_dict[0] = [0, 0, 0]  # background
colors_dict[2] = [216, 67, 82] # weed

fig, ax = plt.subplots(1, 2)

augmented_img, target = next(iterator)
augmented_img = augmented_img[0]   # First element
augmented_img = augmented_img * 255  # denormalize
augmented_img.shape

target = np.array(target[0, ..., 0])   # First element (squeezing channel dimension)

print(np.unique(target))

# Assign colors (just for visualization)
target_img = np.zeros([target.shape[0], target.shape[1], 3])

target_img[np.where(target == 0)] = colors_dict[0]
target_img[np.where(target == 1)] = colors_dict[1]
target_img[np.where(target == 2)] = colors_dict[2]

ax[0].imshow(np.uint8(augmented_img))
ax[1].imshow(np.uint8(target_img))

plt.show()

# MODEL DEFINITION

In [ ]:
vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
vgg.summary()
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
def create_model(depth, start_f, num_classes):

    model = tf.keras.Sequential()
    
    # Encoder
    # -------
    model.add(vgg)
    
    start_f = 256
        
    # Decoder
    # -------
    for i in range(depth):
        model.add(tf.keras.layers.UpSampling2D(2, interpolation='bilinear'))
        model.add(tf.keras.layers.Conv2D(filters=start_f,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))
        model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    # ----------------
    model.add(tf.keras.layers.Conv2D(filters=num_classes,
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='softmax'))
    
    return model

In [ ]:
model = create_model(depth=5, 
                     start_f=8, 
                     num_classes=3)

# Visualize created model as a table
model.summary()

# PARAMS

In [ ]:
# Optimization params
# -------------------

# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy() 
# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Here we define the intersection over union for each class in the batch.
# Then we compute the final iou as the mean over classes
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

# Validation metrics
# ------------------
metrics = ['accuracy', meanIoU]
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# CALLBACKS

In [ ]:
from datetime import datetime

cwd = '/content/drive/MyDrive/'

experiments_dir = os.path.join(cwd, 'AN2DL_experiments_project_2')
if not os.path.exists(experiments_dir):
    os.makedirs(experiments_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'Proj2'

proj_dir = os.path.join(experiments_dir, model_name + '_' + str(now))
if not os.path.exists(proj_dir):
    os.makedirs(proj_dir)
    
callbacks = []

ckpt_dir = os.path.join(proj_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True) 
callbacks.append(ckpt_callback)

tb_dir = os.path.join(proj_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)
callbacks.append(tb_callback)

early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

# TRAINING

In [ ]:
model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_img_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen), 
          callbacks=callbacks)

# SUBMISSION

## MODEL RELOADING

In [ ]:
model.load_weights("/content/drive/MyDrive/AN2DL_experiments_project_2/Proj2_Dec12_14-58-22/ckpts/cp_08.ckpt")

## RLE ENCODE SCRIPT

In [ ]:
def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## SUBMISSION JSON

In [ ]:
def predict_cropped_image(cropped_img):
  out_sigmoid = model.predict(x=tf.expand_dims(cropped_img, 0))
  predicted_class = tf.argmax(out_sigmoid, -1)
  return np.squeeze(predicted_class.numpy())


In [ ]:
import json

testing_dir = '/content/Development_Dataset/Test_Dev'
submission_dict = {}

DIM = 256

for team in os.listdir(testing_dir):
  update_last_row = False
  update_last_column = False
  for crop in os.listdir(os.path.join(testing_dir, team)):
      for f in os.listdir(os.path.join(testing_dir, team, crop, 'Images')):

        img_arr = cv2.imread(os.path.join(testing_dir, team, crop, 'Images', f))
        
        # necessary due to cv2.imread that creates a numpy array 
        # of the BGR image and not of the RGB one
        img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)

        if not update_last_row and img_arr.shape[0]%DIM:
          update_last_row = True
        if not update_last_column and img_arr.shape[1]%DIM:
          update_last_column = True       

        # predictions di default
        total_prediction = np.zeros(img_arr.shape[0:2])
        for l in range(img_arr.shape[0]//DIM):
          for c in range(img_arr.shape[1]//DIM):
            total_prediction[l*DIM:l*DIM+DIM, c*DIM:c*DIM+DIM] = predict_cropped_image(img_arr[l*DIM:l*DIM+DIM, c*DIM:c*DIM+DIM])

        # predictions dell'ultima colonna
        if update_last_column:
          for l in range(img_arr.shape[0]//DIM):
            total_prediction[l*DIM:l*DIM+DIM, img_arr.shape[1]-DIM:img_arr.shape[1]] = predict_cropped_image(img_arr[l*DIM:l*DIM+DIM, img_arr.shape[1]-DIM:img_arr.shape[1]])

        # predictions dell'ultima riga
        if update_last_row:
          for c in range(img_arr.shape[1]//DIM):
            total_prediction[img_arr.shape[0]-DIM:img_arr.shape[0], c*DIM:c*DIM+DIM] = predict_cropped_image(img_arr[img_arr.shape[0]-DIM:img_arr.shape[0], c*DIM:c*DIM+DIM])

        # prediction cella rimanente
        if update_last_column and update_last_row:
          total_prediction[img_arr.shape[0]-DIM:img_arr.shape[0], img_arr.shape[1]-DIM:img_arr.shape[1]] = predict_cropped_image(img_arr[img_arr.shape[0]-DIM:img_arr.shape[0], img_arr.shape[1]-DIM:img_arr.shape[1]])

        img_name = f[:-4]

        submission_dict[img_name] = {}
        submission_dict[img_name]['shape'] = total_prediction.shape
        submission_dict[img_name]['team'] = team
        submission_dict[img_name]['crop'] = crop
        submission_dict[img_name]['segmentation'] = {}

        # RLE encoding
        # crop
        rle_encoded_crop = rle_encode(total_prediction == 1)
        # weed
        rle_encoded_weed = rle_encode(total_prediction == 2)

        submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
        submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed


with open(os.path.join('/content/submission.json'), 'w') as f:
  json.dump(submission_dict, f)